# SentinelHub Downloader:

Library: [Sentinelhub](https://sentinelhub-py.readthedocs.io/en/latest/index.html)

This notebook provides a script for downloading datasets from SentHub, a platform used for managing various datasets.

In this notebook, we explore how to access and download satellite imagery from **Sentinel Hub** using its powerful [API](https://dataspace.copernicus.eu/analyse/apis). By leveraging Sentinel Hub’s capabilities, we can retrieve data from these missions and process it to meet specific analysis requirements, making it a useful tool for environmental monitoring, geospatial analysis, and research applications.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Import Necessary Libraries

In [ ]:
from sentinelhub import (
    SHConfig,
    DataCollection,
    SentinelHubCatalog,
    SentinelHubRequest,
    SentinelHubStatistical,
    BBox,
    bbox_to_dimensions,
    CRS,
    MimeType,
    Geometry,
    MosaickingOrder,
)

import datetime
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets

import rasterio
from rasterio.transform import from_bounds
import leafmap

from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

## Manage Working Directories

In [ ]:
# The directories (if doesn't exist) will be created in the Data folder
download_folder = r".\data\sentinelhub\download"

if not os.path.exists(download_folder):
    os.makedirs(download_folder)

### API and Authentication


In [ ]:
# # Only run this cell if you have not created a configuration.
# config = SHConfig()
# config.sh_client_id = '**-********-****-****-****-************'
# config.sh_client_secret = '********************************'

# config.sh_base_url = 'https://sh.dataspace.copernicus.eu'
# config.sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'

# config.save("cdse")

In [ ]:
config = SHConfig("cdse")

## Define parameters for download

In [ ]:
# Define the bounding box extents in CRS: WGS84 in this format [lon_min, lat_min, lon_max, lat_max]

bbox_wgs84 = [9.1170, 47.6330, 9.2180, 47.7160] # Bounding box for Konstanz, Germany
resolution = 10 # Primary bands of Sentilen2 have spatial resolution of 10m
# Read more at https://docs.sentinel-hub.com/api/latest/data/sentinel-2-l1c/

aoi_bbox = BBox(bbox=bbox_wgs84, crs=CRS.WGS84)
aoi_size = bbox_to_dimensions(aoi_bbox, resolution=resolution)

In [ ]:
initial_date = widgets.DatePicker(description="Select initial date: ",
                                  style=dict(description_width='initial'),
                                  disabled=False,
                                  value=datetime.date(2024,1,1))
initial_date

In [ ]:
final_date = widgets.DatePicker(description="Select final date: ",
                                style=dict(description_width='initial'),
                                disabled=False,
                                value=datetime.date.today())
final_date

In [ ]:
time_interval = (str(initial_date.value), str(final_date.value))

## Retrieve available dataset from SHcatalog

In [ ]:
catalog = SentinelHubCatalog(config=config)

search_iterator = catalog.search(
    DataCollection.SENTINEL2_L2A,
    bbox=aoi_bbox,
    time=time_interval,
    filter=f"eo:cloud_cover <= {20}",
    fields={"include": ["id",
                        "properties.datetime",
                        "properties.eo:cloud_cover",
                        "properties.platform",
                        ],
            "exclude": []},
)

results = list(search_iterator)
print("Total number of results:", len(results))

data = []
for entry in results:
    id = entry['id']
    dt = entry['properties']['datetime']
    date, time = dt.split('T')
    time = time.replace('Z', '')
    cloud_cover = entry['properties']['eo:cloud_cover']
    data.append({#'Id':id,
                 'Date':date, 
                 'Time':time,
                 'Cloud Cover (%)':cloud_cover})

df = pd.DataFrame(data)
df

## Define Evaluation scripts for access and download

In [ ]:
evalscript_sentinel2_all_bands = """
    //VERSION=3
    function setup() {
        return {
            input: [{
                bands: ["B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B10","B11","B12"],
                units: "DN"
            }],
            output: {
                bands: 13,
                sampleType: "INT16"
            }
        };
    }
    function evaluatePixel(sample) {
        return [sample.B01,
                sample.B02,
                sample.B03,
                sample.B04,
                sample.B05,
                sample.B06,
                sample.B07,
                sample.B08,
                sample.B8A,
                sample.B09,
                sample.B10,
                sample.B11,
                sample.B12];
    }
"""

In [ ]:
# Request data with least CC

request_all_bands = SentinelHubRequest(
    data_folder = download_folder,
    evalscript=evalscript_sentinel2_all_bands,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C.define_from(
                "s2l1c", service_url=config.sh_base_url
            ),
            time_interval=time_interval,
            # Mosiac of least cloudy acquisitions
            mosaicking_order=MosaickingOrder.LEAST_CC,
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=aoi_bbox,
    size=aoi_size,
    config=config,
)

all_bands_response = request_all_bands.get_data(save_data=False)

In [ ]:
# Request data using the table "df"
request_all_bands = SentinelHubRequest(
    data_folder=download_folder,
    evalscript=evalscript_sentinel2_all_bands,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=DataCollection.SENTINEL2_L1C.define_from(
                "s2l1c", service_url=config.sh_base_url
            ),
            time_interval='2024-08-23',
        )
    ],
    responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
    bbox=aoi_bbox,
    size=aoi_size,
    config=config,
)

# Get the data
all_bands_response = request_all_bands.get_data(save_data=False)

### Plot true color raster

In [ ]:
from typing import Any, Optional, Tuple

def plot_image(
    image: np.ndarray,
    factor: float = 1.0,
    clip_range: Optional[Tuple[float, float]] = None,
    figsize: Tuple[float, float] = (15, 15),
    **kwargs: Any
) -> None:
    """Utility function for plotting RGB images."""
    fig, ax = plt.subplots(figsize=figsize)
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
image = all_bands_response[0]
min_val = np.percentile(image, 2)  # 2nd percentile
max_val = np.percentile(image, 98)  # 98th percentile
factor = 1.0 / max_val
factor_multiplier = 1.4
print(min_val, max_val)

plot_image(
    image[:, :, [3, 2, 1]],
    factor=factor*factor_multiplier,
    clip_range=(0, 1),
    figsize=(15, 15)
)

### Plot using leafmap

In [ ]:
height, width, bands = image.shape
transform = from_bounds(*bbox_wgs84, width=width, height=height)

rgb_image = image[:, :, [3, 2, 1]]
clipped_image = np.clip(rgb_image * factor * factor_multiplier, 0, 1)
uint8_image = (clipped_image * 255).astype(np.uint8)

i = 0
temp_filename = f"temp_tif_{i:03}.nc"
temp_tif = os.path.join(download_folder, "temp.tif")

while os.path.isfile(temp_tif):
    i += 1
    temp_filename = f"temp_tif_{i:03}.nc"
    temp_tif = os.path.join(download_folder, f"temp_tif_{i:03}.tif")

with rasterio.open(
    temp_tif,
    "w",
    driver="GTiff",
    height=height,
    width=width,
    count=3,  # Only 3 bands for RGB
    dtype=image.dtype,
    crs="EPSG:4326",  # WGS84 coordinate reference system
    transform=transform,
) as dst:
    for i in range(3):  # Loop through the RGB bands
        dst.write(uint8_image[:, :, i], i + 1)

In [ ]:
m = leafmap.Map(center=[(bbox_wgs84[1] + bbox_wgs84[3]) / 2, (bbox_wgs84[0] + bbox_wgs84[2]) / 2], zoom=14)
m.add_raster(
    temp_tif,
    bands=[1,2,3],
)
m

### Plot false color

In [ ]:
plot_image(
    image[:, :, [2, 3, 7]],
    factor=factor*factor_multiplier,
    clip_range=(0, 1),
    figsize=(8, 8)
)

### Plot NDVI

In [ ]:
# Extract the NIR (Band 8) and Red (Band 4) bands
nir = all_bands_response[0][:, :, 7]
red = all_bands_response[0][:, :, 3]

# Compute NDVI (Add a small number to denominator to avoid division by zero)
ndvi = (nir - red) / (nir + red + 1e-10) 

# Plot NDVI
plt.figure(figsize=(8, 8))
cmap = plt.get_cmap('RdYlGn',8)
plt.imshow(ndvi, cmap=cmap, vmin=-1, vmax=1)
plt.colorbar(label='NDVI')
plt.title('NDVI')
plt.axis('off')
plt.show()
